In [ ]:
import xarray as xr

import climepi
import climepi.climdata as climdata
import climepi.epimod as epimod

In [ ]:
ds_clim = climdata.get_example_dataset("isimip_london_small")
ds_clim

In [ ]:
ds_clim_ym = ds_clim.climepi.yearly_average()
ds_clim_ym.climepi.plot_time_series("temperature", by=["scenario", "model"])
# ds_clim.isel(time=ds_clim.time.dt.year==2100).climepi.plot_time_series("temperature", color="r")

In [ ]:
data_vars = ds_clim_ym.climepi.get_non_bnd_data_vars()
poly_coeff_data_vars = [data_var + "_polyfit_coefficients" for data_var in data_vars]
data_var_mapping = dict(zip(poly_coeff_data_vars, data_vars))
fitted_polys = ds_clim_ym[data_vars].polyfit(dim="time", deg=4, full=True)
ds_clim_ym_fit = xr.polyval(
    coord=ds_clim_ym.time,
    coeffs=fitted_polys[poly_coeff_data_vars],
).rename(data_var_mapping)

In [ ]:
fitted_polys

In [ ]:
ds_clim_ym_fit

In [ ]:
p1 = ds_clim_ym_fit.squeeze().climepi.plot_time_series(
    "temperature", by=["scenario", "model"]
)
p2 = ds_clim_ym.squeeze().climepi.plot_time_series(
    "temperature", by=["scenario", "model"]
)
p1 * p2